In [25]:
import pandas as pd
import sys
import csv
import json
from shapely.geometry import shape
import pprint
from flask import Flask
from sqlalchemy import create_engine


In [27]:
stategeo_df = pd.read_csv("Raw Data/state_geojson.csv")
stategeo_df.head()

,type,properties,geometry
0,Feature,"{'GEO_ID': '0400000US23', 'STATE': '23', 'NAME...","{'type': 'MultiPolygon', 'coordinates': [[[[-6..."
1,Feature,"{'GEO_ID': '0400000US25', 'STATE': '25', 'NAME...","{'type': 'MultiPolygon', 'coordinates': [[[[-7..."
2,Feature,"{'GEO_ID': '0400000US26', 'STATE': '26', 'NAME...","{'type': 'MultiPolygon', 'coordinates': [[[[-8..."
3,Feature,"{'GEO_ID': '0400000US30', 'STATE': '30', 'NAME...","{'type': 'Polygon', 'coordinates': [[[-104.057..."
4,Feature,"{'GEO_ID': '0400000US32', 'STATE': '32', 'NAME...","{'type': 'Polygon', 'coordinates': [[[-114.050..."


In [28]:
state = ['Maine', 'Massachusetts', 'Michigan', 'Montana', 'Nevada', 'New Jersey', 'New York', 'North Carolina', 'Ohio', 'Pennsylvania', 'Rhode Island', 'Tennessee', 'Texas', 'Utah', 
'Washington', 'Wisconsin', 'Puerto Rico', 'Maryland', 'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida',
'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Lousiana', 'Minnesota', 'Mississippi', 'Missouri', 'Nebraska', 'New Hampshire', 
'New Mexico', 'North Dakota', 'Oklahoma', 'Oregon', 'South Carolina', 'South Dakota', 'Vermont', 'Virginia', 'West Virginia', 'Wyoming']

In [29]:
stategeo_df['NAME'] = state
stategeo_df.head()

,type,properties,geometry,NAME
0,Feature,"{'GEO_ID': '0400000US23', 'STATE': '23', 'NAME...","{'type': 'MultiPolygon', 'coordinates': [[[[-6...",Maine
1,Feature,"{'GEO_ID': '0400000US25', 'STATE': '25', 'NAME...","{'type': 'MultiPolygon', 'coordinates': [[[[-7...",Massachusetts
2,Feature,"{'GEO_ID': '0400000US26', 'STATE': '26', 'NAME...","{'type': 'MultiPolygon', 'coordinates': [[[[-8...",Michigan
3,Feature,"{'GEO_ID': '0400000US30', 'STATE': '30', 'NAME...","{'type': 'Polygon', 'coordinates': [[[-104.057...",Montana
4,Feature,"{'GEO_ID': '0400000US32', 'STATE': '32', 'NAME...","{'type': 'Polygon', 'coordinates': [[[-114.050...",Nevada


In [30]:
state_df = pd.read_csv("Raw Data/state_data.csv", encoding="utf-8")
state_df.head()

,state,NAME,total_votes20,votes20_Donald_Trump,votes20_Joe_Biden,cases,deaths,TotalPop,VotingAgeCitizen,Men %,...,Hispanic %,White %,Black %,Native %,Asian %,Pacific %,Biden %,Trump %,Republican,Democrat
0,AK,Alaska,260983,148624,102080,16705,83,738565,530553,0.523067,...,0.054854,0.521503,0.012701,0.340637,0.064436,0.005868,0.391137,0.569478,True,False
1,AL,Alabama,2309900,1434159,843473,193985,2973,4850771,3651914,0.484625,...,0.033495,0.661761,0.291164,0.006169,0.007229,0.000182,0.365156,0.620875,True,False
2,AR,Arkansas,1209372,757052,418051,113057,1958,2977944,2183895,0.490826,...,0.051857,0.770472,0.164333,0.005501,0.007049,0.000788,0.345676,0.625988,True,False
3,AZ,Arizona,3366867,1651812,1663447,247473,5979,6809946,4690177,0.497075,...,0.314592,0.517689,0.018469,0.134786,0.013173,0.001290,0.494064,0.490608,False,True
4,CA,California,16092730,5416035,10339137,935878,17671,38982847,24970109,0.496797,...,0.310681,0.565784,0.030425,0.017047,0.073052,0.003010,0.642473,0.336552,False,True


In [31]:
state_df['hispanic_pop'] = round(state_df['TotalPop']*state_df['Hispanic %'])

state_df['white_pop'] = round(state_df['TotalPop']*state_df['White %'])

state_df['black_pop'] = round(state_df['TotalPop']*state_df['Black %'])

state_df['native_pop'] = round(state_df['TotalPop']*state_df['Native %'])

state_df['asian_pop'] = round(state_df['TotalPop']*state_df['Asian %'])

state_df['pacific_pop'] = round(state_df['TotalPop']*state_df['Pacific %'])

In [32]:
merged_df = pd.merge(stategeo_df, state_df, on="NAME")
merged_df.head()
clean_state_df = merged_df.drop(['state'],axis=1)

clean_state_df["cases_per_100k"]= (clean_state_df["cases"]/clean_state_df["TotalPop"])*100000
clean_state_df["deaths_per_100k"]= (clean_state_df["deaths"]/clean_state_df["TotalPop"])*100000


clean_state_df.head()

,type,properties,geometry,NAME,total_votes20,votes20_Donald_Trump,votes20_Joe_Biden,cases,deaths,TotalPop,...,Republican,Democrat,hispanic_pop,white_pop,black_pop,native_pop,asian_pop,pacific_pop,cases_per_100k,deaths_per_100k
0,Feature,"{'GEO_ID': '0400000US23', 'STATE': '23', 'NAME...","{'type': 'MultiPolygon', 'coordinates': [[[[-6...",Maine,786337,341949,420357,6711,147,1330158,...,False,True,19145.0,1279415.0,10504.0,10420.0,10504.0,169.0,504.526530,11.051319
1,Feature,"{'GEO_ID': '0400000US25', 'STATE': '25', 'NAME...","{'type': 'MultiPolygon', 'coordinates': [[[[-7...",Massachusetts,3482216,1131042,2271763,159707,10013,6789319,...,False,True,630365.0,5455488.0,399548.0,20027.0,282388.0,1502.0,2352.327236,147.481655
2,Feature,"{'GEO_ID': '0400000US26', 'STATE': '26', 'NAME...","{'type': 'MultiPolygon', 'coordinates': [[[[-8...",Michigan,5531329,2647004,2795649,197406,7699,9925568,...,False,True,351495.0,8949166.0,393596.0,128261.0,101336.0,1713.0,1988.863509,77.567349
3,Feature,"{'GEO_ID': '0400000US30', 'STATE': '30', 'NAME...","{'type': 'Polygon', 'coordinates': [[[-104.057...",Montana,602781,341765,243716,33495,376,1029862,...,True,False,30486.0,910095.0,2328.0,81715.0,4918.0,319.0,3252.377503,36.509746
4,Feature,"{'GEO_ID': '0400000US32', 'STATE': '32', 'NAME...","{'type': 'Polygon', 'coordinates': [[[-114.050...",Nevada,1337217,635017,671887,101477,1781,2887725,...,False,True,510752.0,2132180.0,64538.0,113809.0,59854.0,6593.0,3514.081154,61.674848


In [33]:
new_df = pd.DataFrame(clean_state_df.geometry.str.split(',',1).tolist(), 
                         columns = ['type','coordinates'])

new_df['type'] = new_df['type'].str.replace('{','') 
new_df['type'] = new_df['type'].str.split(':').str[1]
new_df['type'] = new_df['type'].str.replace("'",'') 

new_df['coordinates'] = new_df['coordinates'].str.replace('}','') 
new_df['coordinates'] = new_df['coordinates'].str.split(':').str[1]


new_df.head()

,type,coordinates
0,MultiPolygon,"[[[[-67.619761, 44.519754], [-67.61541, 44.52..."
1,MultiPolygon,"[[[[-70.832044, 41.606504], [-70.823735, 41.5..."
2,MultiPolygon,"[[[[-88.684434, 48.115785], [-88.675628, 48.1..."
3,Polygon,"[[[-104.057698, 44.997431], [-104.250145, 44...."
4,Polygon,"[[[-114.0506, 37.000396], [-114.049995, 36.95..."


In [34]:
merged_df = pd.merge(new_df, clean_state_df, left_index=True, right_index=True)
final_df = merged_df.drop(['type_y', 'properties', 'geometry'], axis=1)
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 30 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   type_x                49 non-null     object 
 1   coordinates           49 non-null     object 
 2   NAME                  49 non-null     object 
 3   total_votes20         49 non-null     int64  
 4   votes20_Donald_Trump  49 non-null     int64  
 5   votes20_Joe_Biden     49 non-null     int64  
 6   cases                 49 non-null     int64  
 7   deaths                49 non-null     int64  
 8   TotalPop              49 non-null     int64  
 9   VotingAgeCitizen      49 non-null     int64  
 10  Men %                 49 non-null     float64
 11  Women %               49 non-null     float64
 12  Hispanic %            49 non-null     float64
 13  White %               49 non-null     float64
 14  Black %               49 non-null     float64
 15  Native %              49 

In [35]:
geojson = {"type": "FeatureCollection", "features": []}

for _, row in final_df.iterrows():
    feature = {"type": "Feature", "geometry": {"type": row['type_x'], "coordinates": row['coordinates']}, "properties": {"state": row['NAME'], "total_votes20": row['total_votes20'], "votes20_Donald_Trump": row['votes20_Donald_Trump'], "votes20_Joe_Biden": row['votes20_Joe_Biden'], "cases": row['cases'], "deaths": row['deaths'], "TotalPop": row['TotalPop'], 'VotingAgeCitizen': row['VotingAgeCitizen'], "Men %": row['Men %'],
    "Women %": row['Women %'], 'Hispanic %': row['Hispanic %'], 'White %': row['White %'], 'Black %': row['Black %'], 'Native %': row['Native %'], 'Asian %': row['Asian %'], 'Pacific %': row['Pacific %'], 'Biden %': row['Biden %'], 'Trump %': row['Trump %'], 'Republican': row['Republican'], 'Democrat': row['Democrat'], 'hispanic_pop': row['hispanic_pop'], 'white_pop': row['white_pop'], 'black_pop': row['black_pop'], 'native_pop': row['native_pop'], 'asian_pop': row['asian_pop'], 'pacific_pop': row['pacific_pop'], 'cases_per_100k': row['cases_per_100k'], 'deaths_per_100k': row['deaths_per_100k']}

}
    geojson['features'].append(feature)

with open('new.geojson', 'w') as fp:
    (json.dump(geojson, fp)) 



In [36]:
# Create PostgreSQL RDS Database Connection
# conn_string = "postgres:{pwd}@project2.cy7mfzyzcf0j.us-east-2.rds.amazonaws.com:5432/project2db"
# engine = create_engine(f'postgresql://{conn_string}')
# Check table names
# engine.table_names()
# import csv
# with open('state_data.csv') as csvfile:
#     myCSVReader = csv.DictReader(csvfile)
#     change names in placeholder to match names in csv file.
#     sql = """INSERT INTO State(index,type,coordinates,state,total_votes20,votes20_Donald_Trump,votes20_Joe_Biden,cases,deaths,TotalPop,VotingAgeCitizen,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Biden,Trump,Republican,Democrat,hispanic_pop,white_pop,black_pop,native_pop,asian_pop,pacific_pop,cases_per_100k,deaths_per_100k
# )
#           VALUES (%(index)s,%(type)s,%(coordinates)s,%(state)s,%(total_votes20)s,%(votes20_Donald_Trump)s,%(votes20_Joe_Biden)s,%(cases)s,%(deaths)s,%(TotalPop)s, %(VotingAgeCitizen)s,%(Men)s,%(Women)s, %(Hispanic)s,%(White)s,%(Black)s, %(Native)s,%(Asian)s,%(Pacific)s, %(Biden)s,%(Trump)s,%(Republican)s, %(Democrat)s,%(hispanic_pop)s,%(white_pop)s, %(black_pop)s,%(native_pop)s,%(asian_pop)s, %(pacific_pop)s,%(cases_per_100k)s,%(deaths_per_100k)s)"""
#     for row in myCSVReader:
#         use row directly
#         engine.execute(sql, row)
pd.read_sql_query('select * from state', con=engine).head()

,index,type,coordinates,state,total_votes20,votes20_donald_trump,votes20_joe_biden,cases,deaths,totalpop,...,republican,democrat,hispanic_pop,white_pop,black_pop,native_pop,asian_pop,pacific_pop,cases_per_100k,deaths_per_100k
0,0,MultiPolygon,"[[[[-67.619761, 44.519754], [-67.61541, 44.52...",Maine,786337.0,341949.0,420357.0,6711,147,1330158,...,False,True,19145.0,1279415.0,10504.0,10420.0,10504.0,169.0,504.526530,11.051319
1,1,MultiPolygon,"[[[[-70.832044, 41.606504], [-70.823735, 41.5...",Massachusetts,3482216.0,1131042.0,2271763.0,159707,10013,6789319,...,False,True,630365.0,5455488.0,399548.0,20027.0,282388.0,1502.0,2352.327236,147.481655
2,2,MultiPolygon,"[[[[-88.684434, 48.115785], [-88.675628, 48.1...",Michigan,5531329.0,2647004.0,2795649.0,197406,7699,9925568,...,False,True,351495.0,8949166.0,393596.0,128261.0,101336.0,1713.0,1988.863509,77.567349
3,3,Polygon,"[[[-104.057698, 44.997431], [-104.250145, 44....",Montana,602781.0,341765.0,243716.0,33495,376,1029862,...,True,False,30486.0,910095.0,2328.0,81715.0,4918.0,319.0,3252.377503,36.509746
4,4,Polygon,"[[[-114.0506, 37.000396], [-114.049995, 36.95...",Nevada,1337217.0,635017.0,671887.0,101477,1781,2887725,...,False,True,510752.0,2132180.0,64538.0,113809.0,59854.0,6593.0,3514.081154,61.674848
5,5,Polygon,"[[[-75.526844, 39.655713], [-75.526344, 39.65...",New Jersey,4033007.0,1657253.0,2315225.0,239629,16354,8960161,...,False,True,1549783.0,5713328.0,1037401.0,9150.0,650064.0,435.0,2674.382748,182.519042
6,6,MultiPolygon,"[[[[-71.943563, 41.286675], [-71.926802, 41.2...",New York,6647018.0,2852042.0,3701835.0,509735,33535,19798228,...,False,True,1545123.0,16428414.0,1173993.0,75594.0,571194.0,3910.0,2574.649610,169.383846
7,7,MultiPolygon,"[[[[-82.602877, 36.039833], [-82.600741, 36.0...",North Carolina,5474442.0,2736881.0,2663605.0,276692,4383,10052564,...,True,False,726810.0,6980224.0,2066366.0,158746.0,116728.0,3689.0,2752.452011,43.600817
8,8,MultiPolygon,"[[[[-82.813489, 41.723468], [-82.810487, 41.7...",Ohio,5764587.0,3074400.0,2603677.0,219000,5303,11609756,...,True,False,309647.0,10687136.0,483034.0,17352.0,110300.0,2287.0,1886.344554,45.677101
9,9,Polygon,"[[[-75.415041, 39.801786], [-75.428038, 39.80...",Pennsylvania,6804083.0,3332215.0,3385459.0,214416,8803,12790505,...,False,True,543733.0,11457914.0,585200.0,16083.0,187187.0,388.0,1676.368525,68.824491


In [37]:
#final_df.to_csv('state_data.csv')